In [1]:
"""
note
2025.03.13
미니 프로젝트 시작.
상관계수를 직접 구해보는 과정을 해보고 싶었으나 아직 통계와 분석에 대한 이해가 떨어져 바로 결정트리 구현을 진행함.
2025.03.14
넘파이 없이 구현을 목표로 했으나 특징벡터 4개, 클래스 3개로 계산량이 너무 커서 넘파이를 사용해 구현,
추후 상관계수 계산 후 특징벡터 2개를 이용한 이진, 혹은 3 클래스 분류를 진행하고자함.
"""

'\nnote\n2025.03.13\n미니 프로젝트 시작.\n상관계수를 직접 구해보는 과정을 해보고 싶었으나 아직 통계와 분석에 대한 이해가 떨어져 바로 결정트리 구현을 진행함.\n2025.03.14\n넘파이 없이 구현을 목표로 했으나 특징벡터 4개, 클래스 3개로 계산량이 너무 커서 넘파이를 사용해 구현,\n추후 상관계수 계산 후 특징벡터 2개를 이용한 이진, 혹은 3 클래스 분류를 진행하고자함.\n'

In [101]:
from typing import Tuple
import csv
import numpy as np

np.random.seed(0)

In [126]:
f = open("Iris.csv")
rdr = csv.reader(f)

In [127]:
csv_data = [i for i in rdr]
columns = csv_data[0]
data = csv_data[1:]

In [128]:
columns, len(data)

(['Id',
  'SepalLengthCm',
  'SepalWidthCm',
  'PetalLengthCm',
  'PetalWidthCm',
  'Species'],
 150)

In [129]:
set(i[5] for i in data)

{'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'}

In [133]:
_target = {'Iris-versicolor':0, 'Iris-setosa':1, 'Iris-virginica':2}    # Sepal: 꽃받침, Petal: 꽃잎

data = np.array([ [id, *i[1:-1], _target[i[-1]] ] for id, i in enumerate(data) ])
columns = np.array(['Id', 'SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'target'])

data, columns, data.shape

(array([['0', '5.1', '3.5', '1.4', '0.2', '1'],
        ['1', '4.9', '3.0', '1.4', '0.2', '1'],
        ['2', '4.7', '3.2', '1.3', '0.2', '1'],
        ['3', '4.6', '3.1', '1.5', '0.2', '1'],
        ['4', '5.0', '3.6', '1.4', '0.2', '1'],
        ['5', '5.4', '3.9', '1.7', '0.4', '1'],
        ['6', '4.6', '3.4', '1.4', '0.3', '1'],
        ['7', '5.0', '3.4', '1.5', '0.2', '1'],
        ['8', '4.4', '2.9', '1.4', '0.2', '1'],
        ['9', '4.9', '3.1', '1.5', '0.1', '1'],
        ['10', '5.4', '3.7', '1.5', '0.2', '1'],
        ['11', '4.8', '3.4', '1.6', '0.2', '1'],
        ['12', '4.8', '3.0', '1.4', '0.1', '1'],
        ['13', '4.3', '3.0', '1.1', '0.1', '1'],
        ['14', '5.8', '4.0', '1.2', '0.2', '1'],
        ['15', '5.7', '4.4', '1.5', '0.4', '1'],
        ['16', '5.4', '3.9', '1.3', '0.4', '1'],
        ['17', '5.1', '3.5', '1.4', '0.3', '1'],
        ['18', '5.7', '3.8', '1.7', '0.3', '1'],
        ['19', '5.1', '3.8', '1.5', '0.3', '1'],
        ['20', '5.4', '3.4', '

In [123]:
"""
2025.03.13
상관계수를 직접 구해보는 과정을 해보고 싶었으나 아직 통계와 분석에 대한 이해가 떨어져 바로 결정트리 구현을 진행함.
"""

'\n2025.03.13\n상관계수를 직접 구해보는 과정을 해보고 싶었으나 아직 통계와 분석에 대한 이해가 떨어져 바로 결정트리 구현을 진행함.\n'

In [134]:
# np.random.shuffle(data): 원본 변경
# np.random.permutation(data): 원본 유지, 수정본 반환
# 데이터의 클래스 비율을 유지 할 수 있었으나 이후 랜덤포레스트 구현 시 발생 할 차이를 보기 위해 랜덤으로 유지함.

def train_test_split(data, ratio=0.7)-> Tuple[list, list]:
    shuffled_data = np.random.permutation(data)
    index = int(len(data)*ratio)
    return shuffled_data[:index], shuffled_data[index:]

In [177]:
trainX, trainY = train_test_split(data)
trainX.shape, trainY.shape

((105, 6), (45, 6))

In [ ]:
"""
결정트리분류 모델 구현
손실함수가 필요 없는 모델이며 지니지수를 이용해 데이터를 분류한다.
분류에는 2~4 특징을 이용한다. 성능 평가는 ACC(정확도)를 사용한다.
연결리스트, 노드를 사용한다.
attribute:
DB = Decision_Boundary(결정경계) - 모델로 구하고자 하는 목표, 데이터를 분류하는 기준.
maxG = maxG가 작아지도록 결정경계를 설정한다. 기존값은 1. 결정경계를 생성하면서 점점 낮아진다.

DH = Decision_History - 트리가 클래스를 분류한 기록
"""

'\n결정트리분류 모델 구현\n손실함수가 필요 없는 모델이며 지니지수를 이용해 데이터를 분류한다.\n분류에는 2~4 특징을 이용한다. 성능 평가는 ACC(정확도)를 사용한다.\n연결리스트, 노드를 사용한다.\nattribute:\nDB = Decision_Boundary(결정경계) - 모델로 구하고자 하는 목표, 데이터를 분류하는 기준.\n\nmaxG = maxG가 작아지도록 결정경계를 설정한다. 기존값은 1. 결정경계를 생성하면서 점점 낮아진다.\n'

In [ ]:
class DecisionTreeClassifier:
    def __init__(self, depth = 2):
        self.data = None
        self.depth = depth
        self.xLeaf = (3**(depth+1)-2)//2    # 리프 노드를 제외한 노드의 수
        self.nNode = (3**(depth+1)-1)//2    # 리프 노드를 포함한 노드의 수
        self.DB = np.array([[]]).reshape(-1, self.xLeaf)
        # self.history = np.array([]).reshape(self.nNode, -1)
        # self.history = np.repeat([[[1],[None]]], self.nNode, axis=0) # (self.nNode, 2, 1) -> [[서브트리 Gini 지수], [분류된 클래스 인덱스]]
    
    # 서브트리 루트 노드를 받아 최적의 결정경계 산출
    # 입력으로 루트 노드의 데이터 인덱스(Id)를 받음
    def CART(self, _id):
        x = self.data[_id].view()                                          # 데이터 불러오기기
        _cnt = np.bincount(x[:, -1])                                       # x GT 요소별 합 _cnt
        _maxG = 0
        _bd = None

        for i in range(1, self.data.shape[1]-1):                           # 특징벡터 만큼 반복
            _x = x[np.argsort(x[:, i])]                                    # 단일 특징을 기준으로 정렬된 배열 _x
            
            for _idx in range(_x.shape[0]):
                t1 = np.bincount(_x[:_idx, -1].view()) / _cnt
                t1 = np.sum(t1**2)
                t2 = np.bincount(_x[_idx:, -1].view()) / _cnt
                t2 = np.sum(t2**2)
                _G = t1 + t2

                if (_G > _maxG):
                    _maxG = _G
                    if(_idx == 0):
                        _bd = None
                    else:
                        _bd = [i, np.mean(_x[_idx-1:_idx, i]), int(t1 < t2)]       # [,,0] 0이면 t1, 1이면 t2를 한번 더 분할

        return _bd

    def fit(self, x):
        self.data = x
        _id = list(range(self.data.shape[0]))
        # self.history[0, 1] = np.arange(x.shape[0])

        for _node in range(self.xLeaf):
            self.CART(_node)        #


        history = ""

        return history


In [137]:
np.arange(10).dtype

dtype('int32')

In [155]:
a = np.array([[]]).reshape(13,-1)
a[0] = 1
print(a)

[]


In [ ]:
a = np.repeat([[None]], 3, axis=0)
a[1] = 2
a[1].
a[1]

array([4], dtype=object)

In [184]:
#tt = np.repeat([[[1],[None]]], 10, axis=0)
tt = np.arange(30).reshape(-1,2)
tt[0, 1] = 10
tt.shape

(15, 2)

In [185]:
_index = [0,4,6,2]
tt[_index]

array([[ 0, 10],
       [ 8,  9],
       [12, 13],
       [ 4,  5]])

In [186]:
list(range(tt.shape[0]))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [187]:
tt.shape

(15, 2)

In [233]:
import numpy as np

# (5, 3) 형태의 NumPy 배열 (예제 데이터)
arr = np.array([
    [1, 2, 0],
    [2, 5, 1],
    [3, 1, 2],
    [4, 7, 0],
    [5, 6, 0],
    [6, 3, 1],
    [7, 2, 2],
    [8, 1, 0],
])

# 정렬 기준이 되는 (n,1) 열 선택 (예: 두 번째 열 기준으로 정렬)
column_index = 1  # 기준 열 인덱스
sorted_indices = np.argsort(arr[:, column_index])  # 기준 열을 정렬한 인덱스

# 정렬된 배열
sorted_arr = arr[sorted_indices]

print(sorted_arr)
arr1 = arr[np.argsort(arr[:, 1])][:, [1, -1]]

[[3 1 2]
 [8 1 0]
 [1 2 0]
 [7 2 2]
 [6 3 1]
 [2 5 1]
 [5 6 0]
 [4 7 0]]


In [ ]:
t0 = np.bincount(arr1[:, -1])
for _idx in range(arr1.shape[0]):                             
    t1 = np.bincount(arr1[:_idx, -1], minlength=3) / t0
    t2 = np.bincount(arr1[_idx:, -1], minlength=3) / t0
np.add(t1**2, t2**2)

array([0.625, 1.   , 1.   ])

In [228]:
np.bincount(arr[:,-1]), arr[:,-1]
arr[:_idx, -1].view(), arr[_idx:, -1].view()

(array([3, 6, 8, 5]), array([2]))

In [247]:
t1 = 1; t2 = 2
int(t1 > t2)

0